# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

import warnings
warnings.filterwarnings("ignore")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("output_data/clean_city_data.csv")
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nikolskoye,59.7035,30.7861,49.39,73,2,10.16,RU,1.633565e+09
1,provideniya,64.3833,-173.3000,41.11,81,75,6.71,RU,1.633565e+09
2,rikitea,-23.1203,-134.9692,74.26,78,99,15.95,PF,1.633565e+09
3,new norfolk,-42.7826,147.0587,56.71,48,73,5.99,AU,1.633565e+09
4,castro,-24.7911,-50.0119,64.33,94,98,1.10,BR,1.633565e+09
...,...,...,...,...,...,...,...,...,...
535,slave lake,55.2834,-114.7690,44.94,70,100,10.98,CA,1.633566e+09
536,daru,-9.0763,143.2092,81.05,80,100,12.44,PG,1.633566e+09
537,san cristobal,7.7669,-72.2250,71.78,96,70,3.09,VE,1.633566e+09
538,ketchikan,55.3422,-131.6461,52.27,40,20,0.00,US,1.633566e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_data[["Lat","Lng"]].astype(float)
humidity = city_data["Humidity"]
humidity.max()

100

In [4]:
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nikolskoye,59.7035,30.7861,49.39,73,2,10.16,RU,1.633565e+09
1,provideniya,64.3833,-173.3000,41.11,81,75,6.71,RU,1.633565e+09
2,rikitea,-23.1203,-134.9692,74.26,78,99,15.95,PF,1.633565e+09
3,new norfolk,-42.7826,147.0587,56.71,48,73,5.99,AU,1.633565e+09
4,castro,-24.7911,-50.0119,64.33,94,98,1.10,BR,1.633565e+09
...,...,...,...,...,...,...,...,...,...
535,slave lake,55.2834,-114.7690,44.94,70,100,10.98,CA,1.633566e+09
536,daru,-9.0763,143.2092,81.05,80,100,12.44,PG,1.633566e+09
537,san cristobal,7.7669,-72.2250,71.78,96,70,3.09,VE,1.633566e+09
538,ketchikan,55.3422,-131.6461,52.27,40,20,0.00,US,1.633566e+09


In [5]:
fig_layout = {'width': '800px',
              'height': '400px',
              'border': '1px solid black',
              'padding': '1px',
              'margin': '0 auto 0 auto'
             }
fig = gmaps.figure(layout=fig_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=humidity.max(), point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Cloudiness Less than 5.
# Humidity Less than 50.
hotel_df = city_data.loc[(city_data['Max Temp']<80)&(city_data['Max Temp']>70)&(city_data['Wind Speed']<10)&(city_data['Cloudiness']<5)&(city_data['Humidity']<50),:]
hotel_df.index

Int64Index([84, 238, 365, 383, 447, 465, 515, 526], dtype='int64')

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df['Hotel Name']=""

In [8]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
          'radius':5000,
          'types':'hotel',
          'key':g_key,    
}

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    params['location']=f'{lat},{lng}'
    name = requests.get(base_url, params = params).json()
    try:
        hotel_df.loc[index,'Hotel Name'] = name['results'][1]['name']
        print(f"Searching for hotel in city {row['City']}")
    except:
        print('Missing result...skipping.')

Searching for hotel in city silver city
Searching for hotel in city marrakesh
Searching for hotel in city fort morgan
Searching for hotel in city aswan
Searching for hotel in city awbari
Searching for hotel in city ellenabad
Searching for hotel in city tabas
Searching for hotel in city winslow


In [9]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
84,silver city,32.7701,-108.2803,73.36,36,1,8.99,US,1.633566e+09,Copper Manor Motel
238,marrakesh,31.6315,-8.0083,73.47,49,0,8.05,MA,1.633566e+09,Novotel Marrakech Hivernage
365,fort morgan,40.2503,-103.8000,78.22,18,1,9.22,US,1.633566e+09,NAPA Auto Parts - R&K Auto
383,aswan,24.0934,32.9070,74.80,32,0,9.37,EG,1.633566e+09,Nuba Nile
447,awbari,26.5921,12.7805,72.61,27,0,4.94,LY,1.633566e+09,شركة الطارق المتميزة لاستيراد وبيع المواد الغذ...
465,ellenabad,29.4500,74.6500,79.99,47,0,3.38,IN,1.633566e+09,Chachan Computer
515,tabas,33.5959,56.9244,76.30,17,4,1.88,IR,1.633566e+09,کافی نت طبس آنلاین
526,winslow,35.0242,-110.6974,73.58,49,1,0.00,US,1.633566e+09,Best Western Plus Winslow Inn


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

['\n<dl>\n<dt>Name</dt><dd>Copper Manor Motel</dd>\n<dt>City</dt><dd>silver city</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Novotel Marrakech Hivernage</dd>\n<dt>City</dt><dd>marrakesh</dd>\n<dt>Country</dt><dd>MA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>NAPA Auto Parts - R&K Auto</dd>\n<dt>City</dt><dd>fort morgan</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Nuba Nile</dd>\n<dt>City</dt><dd>aswan</dd>\n<dt>Country</dt><dd>EG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>شركة الطارق المتميزة لاستيراد وبيع المواد الغذائية</dd>\n<dt>City</dt><dd>awbari</dd>\n<dt>Country</dt><dd>LY</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Chachan Computer</dd>\n<dt>City</dt><dd>ellenabad</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>کافی نت طبس آنلاین</dd>\n<dt>City</dt><dd>tabas</dd>\n<dt>Country</dt><dd>IR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Best Western Plus Winslow Inn</dd>\n<dt>City</dt><dd>winslow</dd>\n<dt>Cou

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…